### Data Inspection

This notebook is used for the visualization and initial analysis of the EIS measurements

In [ ]:
%matplotlib widget 

from modules import dataset_manipulation as dm
from modules import eisplot as eisplot

import numpy as np
import pandas as pd

cm = 1 / 2.54  # centimeters in inches

## if you have installed latex and want to use it for plots, uncomment the following 3 lines
# eisplot.mpl.rcParams.update({"text.usetex": True,'savefig.format':'pdf'})
# eisplot.mpl.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
# eisplot.mpl.rc('text.latex', preamble=r'\usepackage{underscore}')

## safe figures e.g. with:
# plot_name = "custom_3D_plot"
# eisplot.plt.savefig(r"./figures/" + name_of_this_run + "_" + plot_name + ".pdf")
# eisplot.plt.savefig(r"./figures/" + name_of_this_run + "_" + plot_name + ".png", dpi=600)

The variable ```name_of_this_run``` is used to save and load the data

In [ ]:
name_of_this_run = "example_data"

In [ ]:
destination_filepath = r"./data/eis_datasets/" + name_of_this_run + ".parquet"
df = pd.read_parquet(destination_filepath)
destination_filepath = r"./data/key_lookup/key_lookup_" + name_of_this_run + ".parquet"
key_lookup_df = pd.read_parquet(destination_filepath)

In [ ]:
frequencies = key_lookup_df["frequency"].to_numpy()
abs_keys = key_lookup_df["EIS_Z_abs"].to_list()
phase_keys = key_lookup_df["EIS_Z_phase"].to_list()
re_keys = key_lookup_df["EIS_Z_Re"].to_list()
im_keys = key_lookup_df["EIS_Z_Im"].to_list()

Analysis of individual cells

In [ ]:
# # grab a cell by its name
cell_name = "LiFun_575166-01_002"
df_cell = df.loc[df.index.get_level_values("cell_ID") == cell_name]
# # or just grab the first one
# df_cell = df[np.in1d(df.index.get_level_values(0), [df.first_valid_index()[0]])]
# # or sample e.g. 5 measurements
# df_cell = df.sample(n=5)

df_cell = df_cell.sort_values("SOC", ascending=True)
df_cell = df_cell.sort_values("Temperature", ascending=True)

There are two functions for typical EIS plots: `plot_nyquist_feature` and `plot_bode_feature`.

In [ ]:
fig, (ax1, ax2), cmap = eisplot.plot_bode_feature(
    df_cell, key_lookup_df, feature="Temperature"
)

In [ ]:
fig, ax, cmap = eisplot.plot_nyquist_feature(
    df_cell, key_lookup_df, feature="Temperature"
)

Example of a custom figure:

In [ ]:
feature = "Temperature"
unit = "°C"

feature_values = df_cell[feature].to_numpy(dtype="float64")
c_min_value = np.min(feature_values)
c_max_value = np.max(feature_values)
norm = eisplot.mpl.colors.Normalize(c_min_value, c_max_value)
cmap = eisplot.mpl.cm.ScalarMappable(norm=norm, cmap=eisplot.mpl.cm.turbo)
colors = cmap.to_rgba(feature_values)

size_scale = 100
size_offset = 10
alpha_scale = 0.75
alpha_offset = 0.25
x = df_cell["Voltage"].to_numpy(dtype="float64")
x = np.transpose(np.tile(x, (len(abs_keys), 1)))
y = df_cell[re_keys].to_numpy(dtype="float64") * 1000
z = df_cell[im_keys].to_numpy(dtype="float64") * -1000
colors = np.repeat(colors, len(abs_keys), axis=0)
sizes = df_cell["SOH"].to_numpy(dtype="float64") / 100
sizes = (
    np.clip(np.transpose(np.tile(sizes, (len(abs_keys), 1)) - 0.7), 0, None)
    * 10
    / 3
    * size_scale
    + size_offset
)
alphas = df_cell["SOC"].to_numpy(dtype="float64") / 100
colors[:, 3] = (
    np.transpose(np.tile(alphas, (len(abs_keys), 1))).reshape(-1) * alpha_scale
    + alpha_offset
)
markers = "o"

fig = eisplot.plt.figure(figsize=(12 * cm, 12 * cm))
ax = fig.add_subplot(1, 1, 1, projection="3d")
cbar = fig.colorbar(cmap, ax=ax, location="left", fraction=0.046, pad=0.04)
cbar.set_label(feature + " in " + unit)
ax.scatter(xs=x, ys=y, zs=z, s=sizes, marker=markers, c=colors)

if eisplot.mpl.rcParams["text.usetex"] == True:
    legend_elements = [
        eisplot.plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoC = 100 \%",
            alpha=1,
            markersize=8,
            linestyle="",
        ),
        eisplot.plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoC = 0 \%",
            alpha=0.25,
            markersize=8,
            linestyle="",
        ),
        eisplot.plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoH = 100 \%",
            alpha=0.5,
            markersize=8,
            linestyle="",
        ),
        eisplot.plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoH = 70 \%",
            alpha=0.5,
            markersize=4,
            linestyle="",
        ),
    ]
    ax.set_xlabel("Voltage in V")
    ax.set_ylabel(r"$\Re(\underline{Z})$ in m$\Omega$")
    ax.set_zlabel(r"$\Im(\underline{Z})$ in m$\Omega$")
else:
    legend_elements = [
        eisplot.plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoC = 100 %",
            alpha=1,
            markersize=8,
            linestyle="",
        ),
        eisplot.plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoC = 0 %",
            alpha=0.25,
            markersize=8,
            linestyle="",
        ),
        eisplot.plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoH = 100 %",
            alpha=0.5,
            markersize=8,
            linestyle="",
        ),
        eisplot.plt.Line2D(
            [0],
            [0],
            marker="o",
            color="dimgray",
            label="SoH = 70 %",
            alpha=0.5,
            markersize=4,
            linestyle="",
        ),
    ]
    ax.set_xlabel("Voltage in V")
    ax.set_ylabel(r"Re(Z) in mΩ")
    ax.set_zlabel(r"Im(Z) in mΩ")

legend_elements = [
    eisplot.plt.Line2D(
        [0],
        [0],
        marker="o",
        color="dimgray",
        label="SoC = 100 %",
        alpha=1,
        markersize=8,
        linestyle="",
    ),
    eisplot.plt.Line2D(
        [0],
        [0],
        marker="o",
        color="dimgray",
        label="SoC = 0 %",
        alpha=0.25,
        markersize=8,
        linestyle="",
    ),
    eisplot.plt.Line2D(
        [0],
        [0],
        marker="o",
        color="dimgray",
        label="SoH = 100 %",
        alpha=0.5,
        markersize=8,
        linestyle="",
    ),
    eisplot.plt.Line2D(
        [0],
        [0],
        marker="o",
        color="dimgray",
        label="SoH = 70 %",
        alpha=0.5,
        markersize=4,
        linestyle="",
    ),
]
ax.legend(handles=legend_elements, loc="upper center", ncol=2, fontsize=8)

Before further plotting, lets get an overview of available parameters

In [ ]:
pd.set_option("display.max_columns", None)
df.head(1)
# pd.set_option('display.max_columns', 20)

In [ ]:
print([column for column in list(df) if "ECM" in column])
print([column for column in list(df) if "DRT" in column])
print([column for column in list(df) if "Bode" in column])
print([column for column in list(df) if "Nyquist" in column])
print(df.columns[0:15].values)

The easiest way is to directly use the pandas plot functions

In [ ]:
df.plot.scatter(
    x="SOC", y="ECM_R0", c="Temperature", colormap="turbo", figsize=(16 * cm, 10 * cm)
)

In [ ]:
fig, axs = eisplot.plt.subplots(13, 13, figsize=(25 * cm, 25 * cm))
pd.plotting.scatter_matrix(
    ax=axs,
    frame=df[
        [
            "Voltage",
            "SOH",
            "ECM_L0",
            "ECM_R0",
            "ECM_CPE1_0",
            "ECM_CPE1_1",
            "ECM_R1",
            "ECM_CPE2_0",
            "ECM_CPE2_1",
            "ECM_R2",
            "ECM_CPE3_0",
            "ECM_CPE3_1",
            "ECM_R3",
        ]
    ],
    diagonal="kde",
)
for ax in axs.flatten():
    ax.xaxis.label.set_rotation(90)
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha("right")
eisplot.plt.tight_layout()

In [ ]:
fig, axs = eisplot.plt.subplots(10, 10, figsize=(25 * cm, 25 * cm))
pd.plotting.scatter_matrix(
    ax=axs,
    frame=df[
        [
            "Voltage",
            "SOH",
            "DRT_Peak_0_tau",
            "DRT_Peak_0_gamma",
            "DRT_Peak_1_tau",
            "DRT_Peak_1_gamma",
            "DRT_Peak_2_tau",
            "DRT_Peak_2_gamma",
            "DRT_Peak_3_tau",
            "DRT_Peak_3_gamma",
        ]
    ],
    diagonal="kde",
)
for ax in axs.flatten():
    ax.xaxis.label.set_rotation(90)
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha("right")
eisplot.plt.tight_layout()

In [ ]:
fig, axs = eisplot.plt.subplots(6, 6, figsize=(20 * cm, 20 * cm))
pd.plotting.scatter_matrix(
    ax=axs,
    frame=df[
        [
            "SOH",
            "Voltage",
            "Bode_Phase_Min_Freq_0",
            "Bode_Phase_Min_Value_0",
            "Bode_Phase_Min_Freq_1",
            "Bode_Phase_Min_Value_1",
        ]
    ],
    diagonal="kde",
)
for ax in axs.flatten():
    ax.xaxis.label.set_rotation(90)
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha("right")
eisplot.plt.tight_layout()

In [ ]:
corr = df[
    [
        "SOH",
        "ECM_R0",
        "DRT_Peak_0_gamma",
        "Bode_Phase_Min_Value_0",
        "Bode_Phase_Min_Value_1",
    ]
].corr()
corr.style.background_gradient(cmap="turbo", vmin=-1, vmax=1)

In [ ]:
# create a subset of the df
df_new = df[
    ["SOC", "Temperature", "ECM_R0", "DRT_Peak_0_tau", "Bode_Phase_Min_Freq_0"]
].copy()
# sort by the absolute scalar correlation of the values
sum_corr = abs(df_new.corr()).sum().sort_values(ascending=True).index.values
df_new = df_new[sum_corr]

# create a custom correlation matrix
g = eisplot.cor_matrix(df_new)

In [ ]:
# Configure the custom correlation matrix figure
eisplot.plt.subplots_adjust(bottom=0.07)
eisplot.plt.subplots_adjust(left=0.1)

g.axes[0, 0].set_ylabel("Normalised\n Probability Density", size=8)
g.axes[0, 0].set_yticklabels([])

name = "SoC in $\%$"
g.axes[4, 0].set_xlabel(name, size=8)

if eisplot.mpl.rcParams["text.usetex"] == True:
    name = "$R_0$ in $\Omega$"
else:
    name = "R_0 in Omega"
g.axes[1, 0].set_ylabel(name, size=8)
g.axes[4, 1].set_xlabel(name, size=8)

name = "Tau of First\n Peak of DRT in s"
g.axes[2, 0].set_ylabel(name, size=8)
g.axes[4, 2].set_xlabel(name, size=8)

name = "Temperature\n in °C"
g.axes[3, 0].set_ylabel(name, size=8)
g.axes[4, 3].set_xlabel(name, size=8)

name = "Frequency of first\n Minimum of the Phase in Hz"
g.axes[4, 0].set_ylabel(name, size=8)
g.axes[4, 4].set_xlabel(name, size=8)

#### Further Bode and Nyquist plots

##### Bode

In [ ]:
fig, ax1 = eisplot.plt.subplots(1, 1, figsize=(12 * cm, 7 * cm))
ax2 = ax1.inset_axes([0.5, 0.5, 0.47, 0.47])
df_cell = df_cell.sort_values("Temperature", ascending=False)
eisplot.plot_bode_feature(
    df_cell, key_lookup_df, feature="Temperature", fig=fig, ax1=ax1, ax2=ax2
)
ax2.remove()
ax1.set_autoscaley_on(True)
ax1.set_yscale("log")

In [ ]:
fig, ax1 = eisplot.plt.subplots(1, 1, figsize=(12 * cm, 7 * cm))
ax2 = ax1.inset_axes([0.5, 0.5, 0.47, 0.47])
eisplot.plot_bode_feature(
    df_cell, key_lookup_df, feature="Temperature", fig=fig, ax1=ax2, ax2=ax1
)
ax2.remove()

In [ ]:
fig, axs = eisplot.plt.subplots(4, 1, sharex=True, figsize=(12 * cm, 12 * cm))
df = df.sort_values("SOC", ascending=True)
df = df.sort_values("Temperature", ascending=False)
eisplot.plot_bode_feature(
    df,
    key_lookup_df,
    feature="Temperature",
    fig=fig,
    ax1=axs[0],
    ax2=axs[1],
    ax1_xlabel=False,
    ax2_xlabel=False,
)
df = df.sort_values("SOC", ascending=True)
eisplot.plot_bode_feature(
    df,
    key_lookup_df,
    feature="SOC",
    fig=fig,
    ax1=axs[2],
    ax2=axs[3],
    ax1_xlabel=False,
    subplots_adjust=False,
)

axs[0].set_autoscaley_on(True)
axs[0].set_yscale("log")
axs[2].set_autoscaley_on(True)
axs[2].set_yscale("log")

##### Nyquist

In [ ]:
fig, axs = eisplot.plt.subplots(
    2, 1, sharey=True, sharex=True, figsize=(12 * cm, 10 * cm)
)

df = df.sort_values("SOC", ascending=True)
df = df.sort_values("Temperature", ascending=False)
eisplot.plot_nyquist_feature(
    df, key_lookup_df, feature="Temperature", fig=fig, ax=axs[0], ax_xlabel=False
)
df = df.sort_values("SOC", ascending=True)
eisplot.plot_nyquist_feature(
    df, key_lookup_df, feature="SOC", fig=fig, ax=axs[1], subplots_adjust=False
)

Let's have a look at the average EIS measurement

In [ ]:
abs_value_mean = df.loc[:, abs_keys].to_numpy(dtype="float64").mean(axis=0)
phase_value_mean = df.loc[:, phase_keys].to_numpy(dtype="float64").mean(axis=0)

impedance_mean = abs_value_mean * np.exp(1j * phase_value_mean)

In [ ]:
fig, ax = eisplot.plt.subplots(1, figsize=(12 * cm, 10 * cm))
ax.plot(np.real(impedance_mean) * 1000, np.imag(impedance_mean) * 1000)
ax.grid()
ax.set_aspect("equal", "box")
if eisplot.mpl.rcParams["text.usetex"] == True:
    ax.set_xlabel(r"$\Re(\underline{Z})$ in m$\Omega$")
    ax.set_ylabel(r"$\Im(\underline{Z})$ in m$\Omega$")
else:
    ax.set_xlabel(r"Re(Z) in mΩ")
    ax.set_ylabel(r"Im(Z) in mΩ")
ax.invert_yaxis()

#### Bode and Nyquist of reduced amout of measurements

In [ ]:
fig, ax, cmap = eisplot.plot_nyquist_feature(
    df_cell, key_lookup_df, feature="Temperature", reduce=True, nr_intervals=8
)

In [ ]:
fig, ax, cmap = eisplot.plot_bode_feature(
    df_cell, key_lookup_df, feature="Temperature", reduce=True, nr_intervals=8
)

#### Bode and Nyquist with highlighted frequencies

In [ ]:
frequencies = key_lookup_df["frequency"].to_numpy()
highlight_freqs = [
    frequencies[0],
    frequencies[5],
    frequencies[14],
    frequencies[30],
    frequencies[36],
]

fig, ax, cmap = eisplot.plot_bode_feature(
    df_cell,
    key_lookup_df,
    "Temperature",
    highlight_freqs=highlight_freqs,
    reduce=True,
    nr_intervals=8,
)

In [ ]:
frequencies = key_lookup_df["frequency"].to_numpy()
highlight_freqs = [
    frequencies[0],
    frequencies[5],
    frequencies[14],
    frequencies[30],
    frequencies[36],
]

fig, axs, cmap = eisplot.plot_nyquist_feature(
    df_cell,
    key_lookup_df,
    "Temperature",
    highlight_freqs=highlight_freqs,
    reduce=True,
    nr_intervals=8,
)

#### Bode and Nyquist with highlighted extrema

In [ ]:
df = df.sort_values("Temperature")

In [ ]:
highlight_df_columns = [
    ["Nyquist_Min_Real_Value_0", "Nyquist_Min_Imag_Value_0"],
    ["Nyquist_Max_Real_Value_0", "Nyquist_Max_Imag_Value_0"],
]

fig, axs, cmap = eisplot.plot_nyquist_feature(
    df,
    key_lookup_df,
    "Temperature",
    reduce=True,
    nr_intervals=9,
    highlight_df_columns=highlight_df_columns,
)

In [ ]:
fig, axs = eisplot.plt.subplots(
    2, 1, sharey=False, sharex=True, figsize=(12 * cm, 10 * cm)
)

highlight_df_columns = [
    ["Bode_Phase_Min_Freq_0", "Bode_Phase_Min_Value_0"],
    ["Bode_Phase_Max_Freq_0", "Bode_Phase_Max_Value_0"],
]

fig, axs, cmap = eisplot.plot_bode_feature(
    df,
    key_lookup_df,
    "Temperature",
    fig=fig,
    ax1=axs[0],
    ax2=axs[1],
    reduce=True,
    nr_intervals=8,
    highlight_df_columns=highlight_df_columns,
)